# Model Playground

In [1]:
# Imports
import json
import os
import torch
import numpy as np
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim

import pretty_midi
from model import Net
from midi_to_piano_roll import midi_to_piano_roll
from loss import blur_loss

import time
import pickle

In [2]:
# Load our checkpoint
ckpt_path = "/media/allentao/One Touch/APS360/ckpts/aug1lr0.0001overfit/checkpoint_epoch121.pt"
state = torch.load(ckpt_path)
model = Net(width = 3, batch_size = 1)
model.load_state_dict(state['state_dict'])

<All keys matched successfully>

In [3]:
# Load our data sample
# song midi
IN_FOLDER = '../data/clean_data/'
song_midi_path = IN_FOLDER + "1_0_song.midi"

# convert to piano roll
song_piano_roll = midi_to_piano_roll(song_midi_path)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
# Run inference
model = model.to(device)
out = model(song_piano_roll.unsqueeze(0).to(device))
out.shape

torch.Size([1, 22705, 128])

In [6]:
out

tensor([[[5.9975e-05, 1.7732e-05, 3.8113e-05,  ..., 2.9938e-05,
          2.8968e-05, 2.7440e-05],
         [6.0743e-05, 4.5981e-05, 3.3111e-05,  ..., 3.0454e-05,
          2.6957e-05, 3.1426e-05],
         [3.5914e-05, 1.7293e-05, 1.1712e-04,  ..., 6.1097e-05,
          2.3617e-05, 3.2780e-05],
         ...,
         [4.7145e-05, 6.2734e-05, 3.1881e-05,  ..., 5.5621e-05,
          8.3512e-05, 3.4640e-05],
         [4.4121e-05, 1.7302e-05, 3.2778e-05,  ..., 2.4677e-05,
          3.3164e-05, 5.6470e-05],
         [4.5655e-05, 1.4988e-05, 2.4936e-05,  ..., 2.8319e-05,
          4.0381e-05, 2.5323e-05]]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)

In [8]:
# Convert piano roll to midi file
from pianoroll_to_midi import pianoroll_to_midi

midi_out = pianoroll_to_midi(out.to("cpu"))
midi_out.save('test.mid')

ValueError: too many values to unpack (expected 2)